In [1]:
print("Samavedam")

Samavedam


In [2]:
%pwd

'/Users/beingrampopuri/Documents/genai-projects/GenAI-Medical-Chatbot/research'

In [3]:
import os
os.chdir("../")

%pwd

'/Users/beingrampopuri/Documents/genai-projects/GenAI-Medical-Chatbot'

In [4]:
# !pip install -r requirements.txt

In [5]:
# !pip install langchain_community

In [6]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

/opt/miniconda3/envs/medical-chatbot/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [7]:
# Extract data from PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [8]:
%pwd

'/Users/beingrampopuri/Documents/genai-projects/GenAI-Medical-Chatbot'

In [9]:
extracted_data = load_pdf_file("data/")

Ignoring wrong pointing object 143 0 (offset 0)
Ignoring wrong pointing object 5352 0 (offset 0)
Ignoring wrong pointing object 6806 0 (offset 0)
Ignoring wrong pointing object 6814 0 (offset 0)
Ignoring wrong pointing object 8164 0 (offset 0)
Ignoring wrong pointing object 8242 0 (offset 0)
Ignoring wrong pointing object 8250 0 (offset 0)
Ignoring wrong pointing object 16313 0 (offset 0)
Ignoring wrong pointing object 16863 0 (offset 0)


In [10]:
extracted_data[:10]

[Document(metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250718005309Z00'00'", 'title': 'The-Gale-Encyclopedia-of-Medicine', 'author': 'Sai Phani Ram Popuri', 'moddate': "D:20250718005309Z00'00'", 'source': 'data/Sample-The-Gale-Encyclopedia-of-Medicine.pdf', 'total_pages': 750, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250718005309Z00'00'", 'title': 'The-Gale-Encyclopedia-of-Medicine', 'author': 'Sai Phani Ram Popuri', 'moddate': "D:20250718005309Z00'00'", 'source': 'data/Sample-The-Gale-Encyclopedia-of-Medicine.pdf', 'total_pages': 750, 'page': 1, 'page_label': '2'}, page_content='The GALEENCYCLOPEDIAof\nMEDICINE\nTHIRD EDITION'),
 Document(metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250718005309Z00'00'", 

In [11]:
# Split the data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                                   chunk_overlap = 20)
    
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print('Length of text chunks: ', len(text_chunks))

Length of text chunks:  6562


In [13]:
# !pip install sentence-transformers==3.3.1
# !pip install huggingface-hub

In [14]:
# Download hugging face embeddings
# Convert text into vector embeddings

from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

/var/folders/cs/8n83c9t54zn1zmz251s2d33w0000gn/T/ipykernel_10015/863854089.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
/opt/miniconda3/envs/medical-chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_result = embeddings.embed_query("I am Phani Ram")
print('Type of query_result: ', type(query_result))
print("Length of query result: ", len(query_result))

Type of query_result:  <class 'list'>
Length of query result:  384


In [17]:
len(query_result)

384

In [18]:
# Pinecone vector embedding

from dotenv import load_dotenv
load_dotenv()

from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"))

index_name = "genai-medical-bot"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "dimension": 384,
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"},
            "metric": "cosine"
        }
    )

In [19]:
from langchain_pinecone import PineconeVectorStore

In [20]:
# Embed each chunk and upsert the embeddings into Pinecode Index
PineconeVectorStore.from_documents(documents = text_chunks,
                                   index_name = index_name,
                                   embedding = embeddings)

In [21]:
# Load existing index from pinecone

vector_store = PineconeVectorStore.from_existing_index(index_name="genai-medical-bot",
 embedding=embeddings)

In [22]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {'k':3})

In [23]:
response = retriever.invoke(input = "What is ADHD?")

In [24]:
response

[Document(id='0abd7eb1-021b-4dde-968e-387708bf5de4', metadata={'author': 'Sai Phani Ram Popuri', 'creationdate': "D:20250718005309Z00'00'", 'creator': 'Preview', 'moddate': "D:20250718005309Z00'00'", 'page': 485.0, 'page_label': '486', 'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'source': 'data/Sample-The-Gale-Encyclopedia-of-Medicine.pdf', 'title': 'The-Gale-Encyclopedia-of-Medicine', 'total_pages': 750.0}, page_content='Attention-deficit/\nHyperactivity disorder\n(ADHD)\nDefinition\nAttention-deficit/hyperactivity disorder (ADHD)\nis a developmental disorder characterized by distract-\nibility, hyperactivity, impulsive behaviors, and the\ninability to remain focused on tasks or activities.\nDescription\nADHD, also known as hyperkinetic disorder\n(HKD) outside of the United States, is estimated to\naffect 3-9% of children, and afflicts boys more often\nthan girls. Although difficult to assess in infancy and'),
 Document(id='12732498-f807-4334-8b65-ff35cb518013', 

In [25]:
# # Mistral AI's LLM

# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain.llms import HuggingFacePipeline

# model_id = "mistralai/Mistral-7B-Instruct-v0.2"  # or Mixtral model

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.float16
# )

# # Create a text generation pipeline
# mistral_pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=500,
#     temperature=0.4,
#     top_p=0.95,
#     do_sample=True,
#     repetition_penalty=1.1
# )

# # Wrap in LangChain's LLM interface
# llm = HuggingFacePipeline(pipeline=mistral_pipe)

In [26]:
load_dotenv()

True

In [27]:
from langchain_openai import OpenAI
llm = OpenAI(temperature = 0.4, max_tokens=500)

In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """
        You are a Medical AI-assistant or a bot for question-answering tasks.
        Use the following pieces of retrieved context to answer the question.
        If you do not know the answer, say I do not know.
        Use 3 sentences maximum and keep the answer professional and concise.

        Find the context below: 
        
        {context}
    """
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [29]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# Known context

response = rag_chain.invoke({'input': 'What is ADHD?'})
print(response['answer'])


System: ADHD is a developmental disorder that is characterized by symptoms such as distractibility, hyperactivity, and impulsive behaviors. It is estimated to affect 3-9% of children and is more common in boys than girls. It can be difficult to diagnose in infancy and early childhood.


In [ ]:
# Out of context question
response = rag_chain.invoke({'input': 'What is Alecrtryut xcferdt?'})
print(response['answer'])



System: I do not know.


In [31]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_community.llms import HuggingFacePipeline

# model_id = "mistralai/Mistral-7B-Instruct-v0.2"  # or LLaMA-2/3 model from Meta

# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # Manual device selection
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
# ).to(device)

# # model = AutoModelForCausalLM.from_pretrained(
# #     model_id,
# #     device_map="auto",  # GPU/CPU automatic mapping
# #     torch_dtype="auto"
# # )

# # Create a transformers pipeline
# hf_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     temperature=0.4
# )

# # Wrap with LangChain LLM
# llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [32]:
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.prompts import ChatPromptTemplate

# system_prompt = (
#     """
#     You are a Medical AI-assistant or a bot for question-answering tasks.
#     Use the following pieces of retrieved context to answer the question.
#     If you do not know the answer, say I do not know.
#     Use 3 sentences maximum and keep the answer professional and concise.

#     Find the context below: 
    
#     {context}
#     """
# )

# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", "{input}")
# ])

# # LLM is the HuggingFacePipeline loaded above
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [33]:
# response = rag_chain.invoke({'input': 'What is ADHD?'})
# print(response['answer'])